# Scraping Twitter Data

1. creating developer account

2. introduction to main functions of Twitter API - json structure of a file + Tweepy

3. making queries

4. downloading user timeline - last 3200 users

5. streaming

6. let's analyze the json data

In [26]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *

In [27]:
# download tweepy
f = open("config.txt", "r")
tokens = []
for line in f:
    tokens.append(line.split()[1])

## access token informations 
access_token1 = tokens[0]
access_token_secret1 = tokens[1]

consumer_key1 = tokens[2]
consumer_secret1 =tokens[3]

In [28]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime

In [29]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [91]:
#Class to read the tweets in streaming
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        sleep(5)
        print(status)
        #returning False in on_error disconnects the stream
        return False

Here we download a bunch of tweets with a set of keywords related to covid ```["coronavirus", "covid"]```

In [ ]:
%%time

OUTPUT_FILENAME = "data/three.json"
stop_condition = 100000



# keywords we may want decide to track 
TRACKING_KEYWORDS = ["coronavirus",'covid']
tweets = 6908
while(True):
    try:
        l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
        stream = Stream(auth=api.auth, listener=l)
    
        stream.filter(
            track=TRACKING_KEYWORDS, 
            is_async=False, 
            languages = ["en"]
        )
    except:
        tweets = l.num_tweets + tweets
        print("Collected tweets: ",tweets)
        print ("Error. Restarting Stream.... Error: ")

Collected tweets:  7027
Error. Restarting Stream.... Error: 
Collected tweets:  7154
Error. Restarting Stream.... Error: 
Collected tweets:  7277
Error. Restarting Stream.... Error: 
Collected tweets:  7420
Error. Restarting Stream.... Error: 
Collected tweets:  7598
Error. Restarting Stream.... Error: 
Collected tweets:  7715
Error. Restarting Stream.... Error: 
Collected tweets:  7834
Error. Restarting Stream.... Error: 
Collected tweets:  7945
Error. Restarting Stream.... Error: 
Collected tweets:  8060
Error. Restarting Stream.... Error: 
Collected tweets:  8170
Error. Restarting Stream.... Error: 
Collected tweets:  8288
Error. Restarting Stream.... Error: 
Collected tweets:  8414
Error. Restarting Stream.... Error: 
Collected tweets:  8538
Error. Restarting Stream.... Error: 
Collected tweets:  8652
Error. Restarting Stream.... Error: 
Collected tweets:  8772
Error. Restarting Stream.... Error: 
Collected tweets:  8894
Error. Restarting Stream.... Error: 
Collected tweets:  9012


In [76]:
tweets

4786

In [84]:
tweets = 0
with open("data/three.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]
    df_tweets = pd.DataFrame.from_records(data)
    tweets = tweets + len(df_tweets.index)
    print(tweets)


5674
